# TruBot testing

In [ ]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
import sys

sys.path.append(str(Path().cwd().parent.parent.resolve()))

In [ ]:
from trulens_eval.keys import *
# from trulens_eval.slackbot import get_or_make_chain, get_answer
from trulens_eval.util import TP
from trulens_eval import Tru
from trulens_eval.tru_feedback import Huggingface

In [ ]:
# Tru().start_dashboard(_dev=True)
# Tru().stop_dashboard()

In [ ]:
thread = Tru().start_evaluator()
# Tru().stop_evaluator()
# Tru().reset_database()

In [ ]:
selectors = [0,1,3,4]
messages = ["Who is Shayak?", "Wer ist Shayak?", "Kim jest Shayak?", "¿Quién es Shayak?", "Was ist QII?", "Co jest QII?"]

def test_bot(selector, question):
    print(selector, question)
    chain = get_or_make_chain(cid=question + str(selector), selector=selector)
    answer = get_answer(chain=chain, question=question)
    return answer

results = []

for s in selectors:
    for m in messages:
        results.append(TP().promise(test_bot, selector=s, question=m))

# Local Model Testing

In [ ]:
from langchain import ConversationChain, HuggingFacePipeline
from transformers import LlamaTokenizer, LlamaForCausalLM, pipeline

tokenizer = LlamaTokenizer.from_pretrained("chavinlo/alpaca-native")

base_model = LlamaForCausalLM.from_pretrained("chavinlo/alpaca-native", device_map='auto')

# Convert the model's parameters to a higher precision data type
base_model = base_model.float()

pipe = pipeline(
    "text-generation",
    model=base_model, 
    tokenizer=tokenizer, 
    max_length=256,
    temperature=0.6,
    top_p=0.95,
    repetition_penalty=1.2
)

local_llm = HuggingFacePipeline(pipeline=pipe)

# We are going to set the memory to go back 4 turns
# window_memory = ConversationBufferWindowMemory(k=4)

In [ ]:
conversation = ConversationChain(
    llm=local_llm, 
    verbose=True
)

In [ ]:
# Setup some feedback functions.

from trulens_eval import tru_feedback
from trulens_eval.tru_db import Record
import numpy as np

hugs = tru_feedback.Huggingface()
openai = tru_feedback.OpenAI()

# Toxicity (of the response):
f_toxic = tru_feedback.Feedback(hugs.not_toxic).on_response()

# Language match (between prompt and response):
f_lang_match = tru_feedback.Feedback(hugs.language_match).on(
    text1="prompt", text2="response"
)

# Question to answer relevance:
f_relevance = tru_feedback.Feedback(openai.relevance).on(
    prompt="input", response="output"
)

# Question to context piece relevance:
#f_qs_relevance = tru_feedback.Feedback(openai.qs_relevance).on(
#    question="input",
#    statement=Record.chain.combine_docs_chain._call.args.inputs.input_documents
#).on_multiple(
#    multiarg="statement", each_query=Record.page_content, agg=np.min
#)

feedbacks=[
    # f_toxic, 
    f_lang_match, 
    # f_relevance, 
    # f_qs_relevance
]

In [ ]:
tc = Tru().Chain(conversation, chain_id="llama-7b-hf-convo", feedbacks=feedbacks)

In [ ]:
res, record = tc.call_with_record("What is the meaning of life?")

In [ ]:
res, record = tc.call_with_record("Who is Nora?")